In [1]:
import pandas as pd
import numpy as np

In [2]:
def CalculoPrioridade(df):
    soma = df.sum()                                                           #Soma as linhas da matrix de Comparação (DataFrame)
    df_norm = pd.DataFrame()                                                  #Cria um DataFrame vazio, onde será disposta a matrix normalizada
    for i in range(len(df)):                                                  #Ciclo for para percorrer a matrix de comparação e dividir pela soma, para normalizar 
        df_norm[i] = df[df.columns[i]]/soma[i]                                #Normalizaão
    prio = df_norm.mean(axis=1)                                               #Cálcula as médias das colunas gerando o autovetor de Prioridade
    df_norm.columns = df.columns                                              #Insere as colunas da matrix passada(df) para a matrix criada(df_norm) 
    df_norm['Prioridade'] = prio                                              #Insere na marix normalizada uma coluna contendo as prioridades
    df_norm.sort_values(by='Prioridade', inplace = True, ascending=False)     #Rankeia a matrix normalizada pela a ordem de prioridade
    return prio, df_norm

In [3]:
def CalculoConsistencia(vetor, df, RI):
    df_temp = pd.DataFrame()                             #Cria um DataFrame vazio, onde será disposta a matrix temporaria
    for i in range(len(df)):                             #Ciclo for para percorrer a matrix de comparação e multiplicar cada elemento pelo autovetor prioridade
        df_temp[i] = df[df.columns[i]]*vetor[i]
    df_temp.columns = df.columns                        #Insere as colunas da matrix passada(df) para a matrix criada(df_temp)
    total = (df_temp.sum(axis=1)/vetor).sum()           #Soma das prioridade 
    n = len(df)
    Lambda = total/n
    CI = (Lambda-n)/(n-1) # CI = índice de consistência
    CR = CI/RI            # CR = Razão de Consistência
    return df_temp, CI,CR

In [4]:
def TratamentoPrio(data):
    matriz = []
    coluna = []
    name = list(data.columns)
    for i in data[name[0]]:
        for j in data[name[0]]:
            if (j == 0):
                coluna.append(0)
            else:
                coluna.append(i/j)
        matriz.append(coluna)
        coluna = []
    df = pd.DataFrame(matriz, index= data.index, columns=data.index)
    return df 

In [5]:
def LerTratarDadosQR(planilha):
    data_mod = pd.read_excel(planilha, sheet_name='Moderado', index_col=0)
    df_mod = TratamentoPrio(data_mod)
    prioridade_mod, matrix_norm_mod = CalculoPrioridade(df_mod)
    data_mar = pd.read_excel(planilha, sheet_name='Marginal', index_col=0)
    df_mar = TratamentoPrio(data_mar)
    prioridade_mar, matrix_norm_mar = CalculoPrioridade(df_mar)
    data_cri = pd.read_excel(planilha, sheet_name='Critico', index_col=0)
    df_cri = TratamentoPrio(data_cri)
    prioridade_cri, matrix_norm_cri = CalculoPrioridade(df_cri)
    return prioridade_mod, prioridade_mar, prioridade_cri

In [6]:
def LerTratarDadosCNC(planilha):
    data_sdai = pd.read_excel(planilha, sheet_name='SDAI', index_col=0)
    df_sdai = TratamentoPrio(data_sdai)
    prioridade_sdai, matrix_norm_sdai = CalculoPrioridade(df_sdai)
    data_proc = pd.read_excel(planilha, sheet_name='PROC', index_col=0)
    df_proc = TratamentoPrio(data_proc)
    prioridade_proc, matrix_norm_proc = CalculoPrioridade(df_proc)
    data_ener = pd.read_excel(planilha, sheet_name='ENER', index_col=0)
    df_ener = TratamentoPrio(data_ener)
    prioridade_ener, matrix_norm_ener = CalculoPrioridade(df_ener)
    data_cola = pd.read_excel(planilha, sheet_name='COLA', index_col=0)
    df_cola = TratamentoPrio(data_cola)
    prioridade_cola, matrix_norm_cola = CalculoPrioridade(df_cola)
    return prioridade_sdai, prioridade_proc, prioridade_ener, prioridade_cola

In [7]:
def MatrixDecisaoFinalQR(peso_criterio, prioridade_mod, prioridade_mar, prioridade_cri):
    matrix_decisao = pd.DataFrame([prioridade_mod, prioridade_mar, prioridade_cri], 
                              index=['Moderado', 'Marginal', 'Critico'])
    matrix_decisao = matrix_decisao.transpose()
    matrix_decisao_final = pd.DataFrame()
    matrix_decisao_final['Moderado'] = matrix_decisao['Moderado']*peso_criterio[0]
    matrix_decisao_final['Marginal'] = matrix_decisao['Marginal']*peso_criterio[1]
    matrix_decisao_final['Critico'] = matrix_decisao['Critico']*peso_criterio[2]
    matrix_decisao_final.fillna(0, inplace=True)
    matrix_decisao_final['Soma'] = matrix_decisao_final['Moderado'] + matrix_decisao_final['Marginal'] + matrix_decisao_final['Critico']
    matrix_final_prioridade = matrix_decisao_final['Soma'].sort_values(ascending = False)
    return matrix_final_prioridade

In [8]:
def MatrixDecisãoFinalCNC(peso_criterio, prioridade_sdai, prioridade_proc, prioridade_ener, prioridade_cola):
    matrix_decisao = pd.DataFrame([prioridade_sdai, prioridade_proc, prioridade_ener, prioridade_cola], 
                              index=['SDAI', 'Processo', 'Energia', 'Colaboradores'])
    matrix_decisao = matrix_decisao.transpose()
    matrix_decisao_final = pd.DataFrame()
    matrix_decisao_final['SDAI'] = matrix_decisao['SDAI']*peso_criterio['SDAI']
    matrix_decisao_final['Processo'] = matrix_decisao['Processo']*peso_criterio['PROCESSO/PROJETO']
    matrix_decisao_final['Energia'] = matrix_decisao['Energia']*peso_criterio['CARGA DE ENERGIA']
    matrix_decisao_final['Colaboradores'] = matrix_decisao['Colaboradores']*peso_criterio['COLABORADOR']
    matrix_decisao_final['Soma'] = matrix_decisao_final['SDAI'] + matrix_decisao_final['Processo'] + matrix_decisao_final['Energia']+ matrix_decisao_final['Colaboradores']
    matrix_final_prioridade = matrix_decisao_final['Soma'].sort_values(ascending = False)
    
    return matrix_final_prioridade

In [9]:
def main ():
    RI_QR = 0.52 #Razão de inconsistência, como a matrix é 3x3, a ordem da marix é 3  
    planilha_QR = 'Planilha de Quantitativo de Riscos de Unidades_teste.xlsx'
    matrix_comp_QR = pd.read_excel(planilha_QR, sheet_name='Hierarquizar Risco', index_col=0)
    prioridade_QR, matrix_norm_QR = CalculoPrioridade(matrix_comp_QR)
    matrix_consQR, CI_QR, CR_QR = CalculoConsistencia(prioridade_QR, matrix_comp_QR, RI_QR)
    prioridade_mod, prioridade_mar, prioridade_cri = LerTratarDadosQR(planilha_QR)
    rank_hosp_QR = MatrixDecisaoFinalQR(prioridade_QR, prioridade_mod, prioridade_mar, prioridade_cri)
    
    RI_CNC = 0.89 #Razão de inconsistência, como a matrix é 4x4, a ordem da marix é 4 
    planilha_CNC = 'Planilha de Ranqueamento de Riscos de Unidades_teste.xlsx'
    matrix_comp_CNC = pd.read_excel(planilha_CNC, sheet_name='Hierarquizar Risco', index_col=0)
    prioridade_CNC, matrix_norm_CNC = CalculoPrioridade(matrix_comp_CNC)
    matrix_cons_CNC, CI_CNC, CR_CNC = CalculoConsistencia(prioridade_CNC, matrix_comp_CNC, RI_CNC)
    prioridade_sdai, prioridade_proc, prioridade_ener, prioridade_cola = LerTratarDadosCNC(planilha_CNC)
    rank_hosp_CNC = MatrixDecisãoFinalCNC(prioridade_CNC, prioridade_sdai, prioridade_proc, prioridade_ener, prioridade_cola)
    return rank_hosp_QR, rank_hosp_CNC, CR_QR, CR_CNC

In [10]:
ranking_hospital_QR, ranking_hospital_CNC, Razao_Consistencia_QR, Razao_Consistencia_CNC = main()

In [11]:
ranking_hospital_QR

UNIDADE
  São Lucas              0.061778
  Memorial               0.058133
  Copa Star              0.043114
  São Marcos             0.038499
  UDI                    0.035990
  Norte D'or             0.034451
  São Rafael             0.029066
  Samer                  0.021942
  Esperança Recife       0.014817
  Clínica São Vicente    0.013279
  Jutta Batista          0.007894
  Aviccena               0.004615
  Morumbi                0.001538
  Alphamed               0.000769
  Criança                0.000769
  Itaim                  0.000000
Name: Soma, dtype: float64

In [12]:
Razao_Consistencia_QR

0.03722565476871801

In [13]:
ranking_hospital_CNC

UNIDADE
  São Lucas              0.141140
  Samer                  0.096128
  São Rafael             0.094756
  Norte D'or             0.090941
  UDI                    0.085737
  São Marcos             0.084652
  Memorial               0.083596
  Esperança Recife       0.081203
  Copa Star              0.076197
  Aviccena               0.062524
  Clínica São Vicente    0.051722
  Morumbi                0.013970
  Alphamed               0.012334
  Jutta Batista          0.011775
  Criança                0.011232
  Itaim                  0.002093
Name: Soma, dtype: float64

In [14]:
Razao_Consistencia_CNC

0.09839300351811663

In [15]:
ranking = pd.DataFrame()
ranking['QR'] = ranking_hospital_QR
ranking['CNC'] = ranking_hospital_CNC
ranking

,QR,CNC
UNIDADE,,
São Lucas,0.061778,0.141140
Memorial,0.058133,0.083596
Copa Star,0.043114,0.076197
São Marcos,0.038499,0.084652
UDI,0.035990,0.085737
Norte D'or,0.034451,0.090941
São Rafael,0.029066,0.094756
Samer,0.021942,0.096128
Esperança Recife,0.014817,0.081203


In [80]:
ranking.sum()

QR     0.366654
CNC    1.000000
dtype: float64

In [82]:
Razao_Consistencia_QR

0.03722565476871801

In [84]:
def Avaliacao(CR):
    if (CR<0,1):
        resp = 'Consistênte'
    else: 
        resp = 'Não é consistênte'
    return resp

In [85]:
resposta = Avaliacao(Razao_Consistencia_QR)
resposta

'Consistência'